In [1]:
import numpy as np
import pandas as pd
import string
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from collections import defaultdict
import math
from sklearn.feature_extraction.text import TfidfVectorizer



Q1.
Google News: Pre-trained word vectors trained on a Google News dataset of 100 billion words.

Wikipedia+Gigaword 5: Pre-trained word vectors trained on a dataset of Wikipedia articles and Gigaword 5 newswire text.

Common Crawl: Pre-trained word vectors trained on a dataset of Common Crawl web crawl data.

GloVe: Pre-trained word vectors trained on a combination of Common Crawl and Wikipedia data.

FastText: Pre-trained word vectors trained on a combination of Common Crawl and Wikipedia data, using subword information.

Ques 2

In [2]:
# import urllib.request

# url = 'https://www.dropbox.com/s/8j58pyfzg16yp9e/glove-twitter-50.txt?dl=1'
# filename = 'glove-twitter-50.txt'
# urllib.request.urlretrieve(url, filename)

In [3]:
import requests
import gensim.downloader as api

# Download the pre-trained word vectors
model = api.load("glove-twitter-50")


Ques 3

Let's choose the following ten words:
1.Football
2.rain
3.book
4.fiction
5.gun
6.cars
7.chicken
8.mobile
9.apple
10.apartment

In [4]:
chosen_words = ["football","rain","book","fiction","wwe","cars","chicken","mobile","apple","apartment"]

Ques 4

In [5]:
for word in chosen_words:
    print(f"Most similar words to '{word}':")
    for similar_word, similarity in model.most_similar(word, topn=3):
        print(f"\t{similar_word}: {similarity:.3f}")
    print()


Most similar words to 'football':
	soccer: 0.933
	sports: 0.899
	basketball: 0.897

Most similar words to 'rain':
	snow: 0.894
	sun: 0.890
	cold: 0.863

Most similar words to 'book':
	books: 0.908
	review: 0.860
	read: 0.856

Most similar words to 'fiction':
	thriller: 0.794
	comics: 0.793
	sci-fi: 0.788

Most similar words to 'wwe':
	wrestlemania: 0.844
	smackdown: 0.832
	tna: 0.816

Most similar words to 'cars':
	trucks: 0.874
	bikes: 0.869
	vehicles: 0.828

Most similar words to 'chicken':
	fried: 0.936
	cheese: 0.925
	steak: 0.900

Most similar words to 'mobile':
	network: 0.873
	apps: 0.835
	google: 0.834

Most similar words to 'apple':
	microsoft: 0.867
	google: 0.834
	samsung: 0.819

Most similar words to 'apartment':
	bedroom: 0.866
	basement: 0.846
	apartments: 0.834



In [6]:
import random
from nltk.corpus import movie_reviews
import nltk
nltk.download('movie_reviews')


documents = [(list(movie_reviews.words(fileid)), category)
             for category in movie_reviews.categories()
             for fileid in movie_reviews.fileids(category)]

random.shuffle(documents)
training_documents = documents[:1500]
testing_documents = documents[1500:]


[nltk_data] Downloading package movie_reviews to
[nltk_data]     C:\Users\mailm\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\movie_reviews.zip.


In [7]:
from nltk.corpus import stopwords
import nltk
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))


def preprocess_words(words):
    words = [word.lower() for word in words if word.isalpha()]
    words = [word for word in words if word not in stop_words]
    return words


training_documents = [(preprocess_words(document), label)
                      for document, label in training_documents]
testing_documents = [(preprocess_words(document), label)
                     for document, label in testing_documents]


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\mailm\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [10]:
from gensim.models import Word2Vec

all_words = [word for document,
             label in training_documents for word in document]
word2vec_model = Word2Vec([all_words], 
                          window=5, min_count=1, workers=4)


In [11]:
import numpy as np


def document_features(document):

    vectors = []
    for word in document:
        if word in word2vec_model.wv:
            vectors.append(word2vec_model.wv[word])
    if len(vectors) == 0:
        return np.zeros(100)
    else:
        return np.mean(vectors, axis=0)


training_features = [document_features(document)
                     for document, label in training_documents]
training_labels = [label for document, label in training_documents]

testing_features = [document_features(document)
                    for document, label in testing_documents]
testing_labels = [label for document, label in testing_documents]


In [12]:
from sklearn.svm import SVC
from sklearn.metrics import classification_report

svm_classifier = SVC(kernel='linear')
svm_classifier.fit(training_features, training_labels)

testing_predictions = svm_classifier.predict(testing_features)

print(classification_report(testing_labels, testing_predictions))


              precision    recall  f1-score   support

         neg       0.00      0.00      0.00       254
         pos       0.49      1.00      0.66       246

    accuracy                           0.49       500
   macro avg       0.25      0.50      0.33       500
weighted avg       0.24      0.49      0.32       500



d:\Program Files(D)\Anaconda\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
d:\Program Files(D)\Anaconda\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
d:\Program Files(D)\Anaconda\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
